### Predecir próximas fechas de gastos recurrentes

In [ ]:
import pandas as pd
# Cargar datasets
df_clientes = pd.read_csv("datos/base_clientes_final.csv")
df_trans = pd.read_csv("datos/base_transacciones_final.csv")


In [3]:
df_trans.head()

,id,fecha,comercio,giro_comercio,tipo_venta,monto
0,91477f382c3cf63ab5cd9263b502109243741158,2022-01-02,AMAZON,COMERCIOS ELECTRONICOS (VTAS POR INTERNET),digital,5.99
1,91477f382c3cf63ab5cd9263b502109243741158,2022-01-05,RAPPI,SERVICIOS EMPRESARIALES - NO CLASIFICADOS,digital,13.01
2,91477f382c3cf63ab5cd9263b502109243741158,2022-01-05,RAPPI,SERVICIOS EMPRESARIALES - NO CLASIFICADOS,digital,15.84
3,91477f382c3cf63ab5cd9263b502109243741158,2022-01-05,AMAZON,COMERCIOS ELECTRONICOS (VTAS POR INTERNET),digital,8.17
4,91477f382c3cf63ab5cd9263b502109243741158,2022-01-05,AMAZON,COMERCIOS ELECTRONICOS (VTAS POR INTERNET),digital,2.54


In [5]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import warnings

warnings.filterwarnings("ignore")  # para silenciar warnings de convergencia

# 1. Carga datos
df = pd.read_csv("datos/base_transacciones_final.csv", parse_dates=["fecha"])

# 2. Preprocesamiento: ordenar y calcular delta_days
df = df.sort_values(["id","comercio","fecha"])
df["delta_days"] = df.groupby(["id","comercio"])["fecha"].diff().dt.days
df_delta = df.dropna(subset=["delta_days"])  # elimina la primera transacción de cada grupo

# 3. Definir función que ajusta ARIMA y predice el próximo Δt
def predecir_delta_una_sola_serie(serie_delta):
    """
    serie_delta: pd.Series indexada por fecha, valores = días entre transacciones
    devuelve: float Δt_predicho
    """
    # Si la serie es muy corta, devolvemos simplemente el promedio
    if len(serie_delta) < 5:
        return serie_delta.mean()
    # Ajustamos ARIMA(1,0,0) como ejemplo; puedes cambiar (p,d,q)
    modelo = ARIMA(serie_delta, order=(1,0,0))
    res = modelo.fit()
    # forecast devuelve un array de length=1
    return float(res.forecast(1))

# 4. Loop sobre todos los grupos para obtener la predicción
resultados = []
for (cliente, comercio), grupo in df_delta.groupby(["id","comercio"]):
    # construimos la serie: índice = fecha, valores = delta_days
    ts = grupo.set_index("fecha")["delta_days"]
    # predecimos el delta
    dt_pred = predecir_delta_una_sola_serie(ts)
    # calculamos la fecha estimada
    fecha_ultimo = grupo["fecha"].max()
    fecha_pred = fecha_ultimo + pd.Timedelta(days=round(dt_pred))
    resultados.append({
        "id": cliente,
        "comercio": comercio,
        "delta_predicho": dt_pred,
        "fecha_ultimo": fecha_ultimo,
        "fecha_proximo_gasto": fecha_pred
    })

df_pred = pd.DataFrame(resultados)

# 5. Guarda o muestra los resultados
df_pred.to_csv("prediccion_recurrencia.csv", index=False)
print(df_pred.head(10))


                                         id      comercio  delta_predicho  \
0  003d9abe467a91847d566cf455bd2d7d6c8f7e75        AMAZON        9.895788   
1  003d9abe467a91847d566cf455bd2d7d6c8f7e75  AMAZON PRIME       27.775597   
2  003d9abe467a91847d566cf455bd2d7d6c8f7e75     CINEPOLIS       44.745551   
3  003d9abe467a91847d566cf455bd2d7d6c8f7e75          DIDI        2.420320   
4  003d9abe467a91847d566cf455bd2d7d6c8f7e75     DIDI FOOD       22.254159   
5  003d9abe467a91847d566cf455bd2d7d6c8f7e75    DIDI RIDES        1.453467   
6  003d9abe467a91847d566cf455bd2d7d6c8f7e75      DIDIFOOD       19.666667   
7  003d9abe467a91847d566cf455bd2d7d6c8f7e75        ITUNES       31.420001   
8  003d9abe467a91847d566cf455bd2d7d6c8f7e75    KUESKI PAY       12.156972   
9  003d9abe467a91847d566cf455bd2d7d6c8f7e75  MERCADO PAGO       21.215824   

  fecha_ultimo fecha_proximo_gasto  
0   2022-08-02          2022-08-12  
1   2023-01-16          2023-02-13  
2   2022-12-28          2023-02-11  
3   

In [7]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import warnings

warnings.filterwarnings("ignore")

# 1. Carga tus datos reales
df = pd.read_csv("datos/base_transacciones_final.csv", parse_dates=["fecha"])

# 2. Ordenar y calcular delta_days
df = df.sort_values(["id","comercio","fecha"])
df["delta_days"] = df.groupby(["id","comercio"])["fecha"].diff().dt.days
df_delta = df.dropna(subset=["delta_days"])

# 3. Función para predecir Δt con ARIMA(1,0,0)
def predict_delta(serie_delta):
    if len(serie_delta) < 5:
        return serie_delta.mean()
    model = ARIMA(serie_delta, order=(1,0,0)).fit()
    return float(model.forecast(1))

# 4. Función para predecir monto con ARIMA sobre serie diaria
def predict_amount(grupo, days_ahead):
    # serie diaria de montos
    ts_amt = grupo.set_index("fecha")["monto"].resample("D").sum().fillna(0)
    # si muy corta, promedio histórico
    if len(ts_amt) < 10 or days_ahead < 1:
        return ts_amt.mean()
    try:
        model = ARIMA(ts_amt, order=(1,1,0)).fit()
        # forecast n pasos y tomamos el último
        fc = model.forecast(steps=days_ahead)
        return float(fc.iloc[-1])
    except:
        return ts_amt.mean()

# 5. Loop sobre cada (id, comercio)
results = []
for (cliente, comercio), grupo in df_delta.groupby(["id","comercio"]):
    # 5.1 predecir delta
    ts_delta = grupo.set_index("fecha")["delta_days"]
    dt_pred = round(predict_delta(ts_delta))
    fecha_ult = grupo["fecha"].max()
    fecha_pred = fecha_ult + pd.Timedelta(days=dt_pred)
    # 5.2 predecir monto en esa fecha
    amt_pred = predict_amount(df[(df.id==cliente)&(df.comercio==comercio)], dt_pred)
    results.append({
        "id": cliente,
        "comercio": comercio,
        "delta_predicho": dt_pred,
        "fecha_ultimo": fecha_ult,
        "fecha_proximo_gasto": fecha_pred,
        "monto_predicho": amt_pred
    })

df_pred = pd.DataFrame(results)
df_pred.to_csv("prediccion_recurrencia_monto.csv", index=False)
print(df_pred.head())


                                         id      comercio  delta_predicho  \
0  003d9abe467a91847d566cf455bd2d7d6c8f7e75        AMAZON              10   
1  003d9abe467a91847d566cf455bd2d7d6c8f7e75  AMAZON PRIME              28   
2  003d9abe467a91847d566cf455bd2d7d6c8f7e75     CINEPOLIS              45   
3  003d9abe467a91847d566cf455bd2d7d6c8f7e75          DIDI               2   
4  003d9abe467a91847d566cf455bd2d7d6c8f7e75     DIDI FOOD              22   

  fecha_ultimo fecha_proximo_gasto  monto_predicho  
0   2022-08-02          2022-08-12       32.458411  
1   2023-01-16          2023-02-13        7.755855  
2   2022-12-28          2023-02-11       31.614555  
3   2022-08-28          2022-08-30        6.929996  
4   2022-10-16          2022-11-07       11.632586  
